# 8.4 대용량 데이터 처리: 온라인 알고리즘과 외부 메모리 학습
* 외부 메모리 학습
    * 데이터셋을 작은 배치(batch) 단위로 나누어 분류기를 점진적으로 학습
    * CountVectorizer 클래스 사용 불가.
        * 전체 어휘 사전을 메모리에 가지고 있어야 하기 때문.
    * TfidfVectorizer 클래스 사용 불가.
        * 역문서 빈도를 계산하기 위해 훈련 데이터셋의 특성 벡터를 모두 메모리에 가지고 있어야 하기 때문
    * 사이킷런 HashingVectorizer 클래스
        * 데이터 종류에 상관없이 사용 가능.
        * 32비트 MurmurHash3 함수를 사용한 해싱 트릭 사용.

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# tokenizer() : 텍스트 데이터를 정제하고 불용어를 제외한 후 단어 토큰으로 분리
import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized


# stream_docs() : 한 번에 한 문서씩 읽어서 반환하는 제너레이터
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # 헤더 넘기기
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [3]:
# movie_data.csv 파일에서 첫번재 문서를 읽는다
next(stream_docs(path='/content/drive/MyDrive/머신러닝 교과서 with 파이썬, 사이킷런, 텐서플로/data/movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [4]:
# get_minibatch() : stream_docs 함수에서 문서를 읽어 size 매개변수에서 지정한 만큼 문서를 반환
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [5]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21, # 특성 개수 -> 크게하면 해시 충돌 가능성을 줄일 수 있지만, 로지스틱 회귀 모델의 가중치 개수도 늘어난다.
                         preprocessor=None, 
                         tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1)

doc_stream = stream_docs(path='/content/drive/MyDrive/머신러닝 교과서 with 파이썬, 사이킷런, 텐서플로/data/movie_data.csv')

In [6]:
!pip install pyprind

In [7]:
# 외부 메모리 학습
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:33


진행 막대(PyPrind.ProgBar) 객체를 45번 반복으로 초기화하고 이어지는 fot 반복문에서 45개의 미니 배치를 반복한다. 각 미니배치를 1,000개의 문서로 구성된다. 

In [8]:
# 점진적인 학습 과정이 끝나면 마지막 5,000개의 문서를 사용하여 모델 성능 평가
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('정확도: %.3f' % clf.score(X_test, y_test))

정확도: 0.868


* 외부 메모리 학습
    * 메모리 효율성이 좋고, 모델 훈련 속도가 빠르다.

In [9]:
# 나머지 5,000개의 문서를 사용하여 모델 업데이트
clf = clf.partial_fit(X_test, y_test)

* 참고
    * word2vec 모델
        * 신경망을 기반으로 한 비지도 학습 알고리즘으로 자동으로 단어 사이의 관계를 학습.
        * 비슷한 의미를 가진 단어를 비슷한 클러스터로 모으는 것.
        * word2vec에서 만든 벡터 공간을 사용하면 모델이 간단한 벡터 연산으로 단어를 생성할 수 있다.
        * https://code.google.com/p/word2vec/

# 8.5 잠재 디리클레 할당을 사용한 토픽 모델링
* 토픽 모델링(topic modeling)
    * 레이블이 없는 텍스트 문서에 토픽을 할당하는 광범위한 분야
    * 카테고리 레이블을 할당
    * 비지도 학습의 클러스터링
    * 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)

### 8.5.1 LDA를 사용한 텍스트 문서 분해
* 여러 문서에 걸쳐 자주 등장하는 단어의 그룹을 찾는 확률적 생성 모델
    * LDA는 Count기반의 Vectorizer만 적용.
    * LDA는 입력 : BoW 모델
        * 입력으로 받은 BoW 모델을 두 개의 행렬로 분해
            * 문서-토픽 행렬
            * 단어-토픽 행렬
    * 분해한 두 행렬을 곱해서 가능한 작은 오차로 BoW 입력 행렬을 재구성할 수 있도록 LDA가 BoW 행렬을 분해한다.
    * 단점 : 미리 토픽 개수를 정해야한다.
        * 토픽 개수는 LDA의 하이퍼 파라미터로, 수동으로 지정.

### 8.5.2 사이킷런 LDA
* 사이킷런 LatentDirichletAllocation 클래스

In [10]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/머신러닝 교과서 with 파이썬, 사이킷런, 텐서플로/data/movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [11]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1, # 최대 문서 빈도 10%로 지정하여 여러 문서에 걸쳐 너무 자주 등장하는 단어를 제외
                        max_features=5000) # 가장 자주 등장하는 단어 5,000개로 단어 수를 제한.
X = count.fit_transform(df['review'].values)

* 자주 등장하는 단어를 제외하는 이유
    * 모든 문서에 걸쳐 등장하는 단어일 수 있다.
    * 이러한 단어는 문서의 특정 토픽 카테고리와 관련성이 적다.
* 자주 등장하는 단어 수 제한하는 이유
    * 데이터셋의 차원을 제한하여 LDA 추론 성능을 향상시킨다.

In [12]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch') # 'batch' : lda 추정기가 한 번 반복할 때 가능한 모든 훈련 데이터(BoW)를 사용하여 학습된다.
                                                         # 'online' 보다 느리지만 더 정확한 결과를 만든다.
X_topics = lda.fit_transform(X)

In [13]:
lda.components_.shape # 열 개의 토픽에 대해 오름차순으로 단어의 중요도를 담은 행렬이 저장된다.

(10, 5000)

In [14]:
# 최상위 5개 단어 출력
n_top_words = 5
feature_names = count.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1] ]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode war episodes tv
Topic 9:
book version original read novel
Topic 10:
action fight guy guys cool


각 토픽에서 가장 중요한 단어 다섯 개를 기반으로 LDA가 다음 토픽을 구별했다고 추측할 수 있다.

1.	대체적으로 형편없는 영화(실제 토픽 카테고리가 되지 못함)
2.	가족 영화
3.	전쟁 영화
4.	예술 영화
5.	범죄 영화
6.	공포 영화
7.	코미디 영화
8.	TV 쇼와 관련된 영화
9.	소설을 원작으로 한 영화
10.	액션 영화

카테고리가 잘 선택됐는지 확인하기 위해 공포 영화 카테고리에서 3개 영화의 리뷰를 출력(공포 영화는 카테고리 6, 인덱스로는 5)

In [15]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\n공포 영화 #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


공포 영화 #1:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

공포 영화 #2:
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

공포 영화 #3:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...
